In [3]:
!pip install jsonpath_ng


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.2 MB/s eta 0:00:00


In [4]:
import os
import csv
import json
import numpy as np
import pandas as pd
import sqlite3
import functools as ft
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from jsonpath_ng import parse
from enum import Enum
%matplotlib inline

### Source to Target Mapping - OOP Approach for Single Table


In [5]:
with open("factTableShorter.csv", "r", encoding="utf-8-sig") as file:
    data = list(csv.DictReader(file))
    for item in data:
        item["TotalPrice"] =str(item["TotalPrice"])
data[:10]

[{'TransactionNo': '581482', 'CustomerNo': '17490', 'TotalPrice': '257.64'},
 {'TransactionNo': '581475', 'CustomerNo': '13069', 'TotalPrice': '383.4'},
 {'TransactionNo': '581475', 'CustomerNo': '13069', 'TotalPrice': '138.36'},
 {'TransactionNo': '581475', 'CustomerNo': '13069', 'TotalPrice': '127.8'},
 {'TransactionNo': '581475', 'CustomerNo': '13069', 'TotalPrice': '71.64'},
 {'TransactionNo': '581475', 'CustomerNo': '13069', 'TotalPrice': '255.6'},
 {'TransactionNo': '581475', 'CustomerNo': '13069', 'TotalPrice': '207.54'},
 {'TransactionNo': '581475', 'CustomerNo': '13069', 'TotalPrice': '147'},
 {'TransactionNo': '581475', 'CustomerNo': '13069', 'TotalPrice': '127.8'},
 {'TransactionNo': '581475', 'CustomerNo': '13069', 'TotalPrice': '253.2'}]

###  Abstract Base Calss (ABC) for mutual methods

In [6]:
class Interface(ABC):

    @abstractmethod
    def get_data_by_field(self, field_name):
        """Fetch the data by given feild name """

    @abstractmethod
    def get_data_by_id(self, id):
        """Fetch the data by given ID  """

    @abstractmethod
    def get(self):
        """Fetch all data """

### Transform Operations
inherithed from Enum - class that automatic enumrate the variables

In [7]:
class TransformMask(Enum):
    # add here any masks you want
    CLEAN_STRING = ".strip().lower()"
    CAPITAL_LETTER = ".strip().lower().title()"


Database Class - Define Common Properties for Source, Target, Mapping

In [9]:
class Database:
    def __init__(self):
        self.db = {
            "source": [],
            "destination": [],
            "transform": [],
            "mapping": []
        }
        #add_source
        self.add_source(1,"TransactionNo","$.TransactionNo","str",True)
        self.add_source(2,"CustomerNo","$.CustomerNo","str",True)
        self.add_source(3,"TotalPrice","$.TotalPrice","str",True)

        #add_transform
        self.add_transform(1,'CLEAN_STRING')


        #add_destination
        self.add_destination(1,"transaction_id","transaction_id","str","n/a")
        self.add_destination(2,"customer_id","customer_id","str","n/a")
        self.add_destination(3,"total_price","total_price","float","0")

        #add_mapping
        self.add_mapping(1,1,1,1)
        self.add_mapping(2,2,2)
        self.add_mapping(3,3,3,2)

    def add_source(self,id,source_field_name,source_field_mapping,source_field_type,is_required):
      if not isinstance(is_required, bool):
        print("Error. Please enter if required.")
      else:
        source = {
            "id": id,
            "source_field_name": source_field_name,
            "source_field_mapping": source_field_mapping,
            "source_field_type": source_field_type,
            "is_required": is_required
        }
        self.db["source"].append(source)

    def add_transform(self,id,transform_mask):
      transform={
          "id": id,
          "transform_mask": transform_mask
      }
      self.db["transform"].append(transform)

    def add_destination(self,id,destination_field_name,destination_field_mapping,destination_field_type,default_value):
        destination = {
              "id": id,
              "destination_field_name": destination_field_name,
              "destination_field_mapping": destination_field_mapping,
              "destination_field_type": destination_field_type,
              "default_value": default_value
          }
        self.db["destination"].append(destination)

    def add_mapping(self,id,mapping_source,mapping_destination,mapping_transform=None):
      if mapping_transform is not None:
        mapping={
          "id": id,
          "mapping_source": mapping_source,
          "mapping_destination": mapping_destination,
          "mapping_transform": mapping_transform
        }
      else:
        mapping={
          "id": id,
          "mapping_source": mapping_source,
          "mapping_destination": mapping_destination
          }
      self.db["mapping"].append(mapping)

    @property
    def get_data_source_target_mapping(self):
        return self.db

## **Source class**
Inherited from Interface for the common methods and from Database for common variables

In [10]:
class Source(Interface, Database):
    def __init__(self):
        Database.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("source")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id") == self.id:
                return x
        return None

## **Target class**
Inherited from Interface for the common methods and from Database for common variables

In [11]:
class Target(Interface, Database):

    def __init__(self):
        Database.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("destination")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

## **Transform Class**
Inherited from Interface for the common methods and from Database for common variables

In [12]:
class Transform(Interface, Database):

    def __init__(self):
        Database.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("transform", [])

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

## **Mapping class**
Inherited from Interface for the common methods and from Database for common variables

In [13]:
class Mappings(Interface, Database):

    def __init__(self):
        Database.__init__(self)

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("mapping")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

    def get_data_by_field(self, field_name):
        return None

### Format Class - JSON

Search the source data value inside a JSON file

In [14]:
class JsonQuery:
    def __init__(self, json_path, json_data):
        self.json_path = json_path
        self.json_data = json_data

    def get(self):
        jsonpath_expression = parse(self.json_path)
        match = jsonpath_expression.find(self.json_data)
        source_data_value = match[0].value
        return source_data_value

### Combine it All - STTM

In [15]:
class STTM:
    def __init__(self, input_json):
        self.input_json = input_json
        self.mapping_instance = Mappings()
        self.source_instance = Source()
        self.destination_instance = Target()
        self.transform_instance = Transform()
        self.look_up_mask = {i.name: i.value for i in TransformMask}
        self.json_data_transformed = {}

    def _get_mapping_data(self):
        return self.mapping_instance.get

    def _get_mapping_source_data(self):
        return self.source_instance.get

    def get_transformed_data(self):

        for mappings in self._get_mapping_data():

            """fetch the source mapping """
            mapping_source_id = mappings.get("mapping_source")
            mapping_destination_id = mappings.get("mapping_destination")
            mapping_transform_id = mappings.get("mapping_transform")

            mapping_source_data = self.source_instance.get_data_by_id(id=mapping_source_id)
            transform_data = self.transform_instance.get_data_by_id(id=mapping_transform_id)

            """Fetch Source  field Name"""
            source_field_name = mapping_source_data.get("source_field_name")

            """if field given is not present incoming json """
            if source_field_name not in self.input_json.keys():
                if mapping_source_data.get("is_required"):
                    raise Exception(
                        "Alert ! Field {} is not present in JSON please FIX mappings ".format(source_field_name))
                else:
                    pass

            else:
                source_data_value = JsonQuery(
                    json_path=mapping_source_data.get("source_field_mapping"),
                    json_data=self.input_json
                ).get()

                """check the data type for source if matches with what we have """
                if mapping_source_data.get("source_field_type") != type(source_data_value).__name__:
                    if source_data_value is not None:
                        _message = (
                            "Alert ! Source Field :{} Datatype has changed from {} to {} ".format(source_field_name,
                                                                                                  mapping_source_data.get(
                                                                                                      "source_field_type"),
                                                                                                  type(
                                                                                                      source_data_value).__name__))
                        print(_message)
                        raise Exception(_message)

                """Query and fetch the Destination | target """
                destination_mappings_json_object = self.destination_instance.get_data_by_id(
                    id=mappings.get("mapping_destination"))
               # print(mappings.get("mapping_destination"))
                destination_field_name = destination_mappings_json_object.get("destination_field_name")
                destination_field_type = destination_mappings_json_object.get("destination_field_type")

                dtypes = [str, float, list, int, set, dict]

                for dtype in dtypes:

                    """Datatype Conversion """
                    if destination_field_type == str(dtype.__name__):

                        """is source is none insert default value"""
                        if source_data_value is None:
                            self.json_data_transformed[destination_field_name] = dtype.__call__(
                                destination_mappings_json_object.get("default_value")
                            )

                        else:
                            """check if you have items to transform"""
                            if transform_data is not None:
                                """ check for invalid mask name """
                                if transform_data.get("transform_mask") not in list(self.look_up_mask.keys()):
                                    raise Exception(
                                        f"Specified Transform {transform_data.get('transform_mask')} is not available please select from following Options :{list(self.look_up_mask.keys())}")
                                else:
                                    mask_apply = self.look_up_mask.get(transform_data.get("transform_mask"))
                                    converted_dtype = dtype.__call__(source_data_value)
                                    mask = f'converted_dtype{mask_apply}'
                                    curated_value = eval(mask)
                                    self.json_data_transformed[destination_field_name] = curated_value

                            else:
                                self.json_data_transformed[destination_field_name] = dtype.__call__(source_data_value)

        return self.json_data_transformed

In [16]:
transformed_data = []
for item in data:
    helper = STTM(input_json=item)
    response = helper.get_transformed_data()
    transformed_data.append(response)
    print(response)

Streaming output truncated to the last 5000 lines.
{'transaction_id': '580680', 'customer_id': '13001', 'total_price': 21.72}
{'transaction_id': '580680', 'customer_id': '13001', 'total_price': 43.44}
{'transaction_id': '580680', 'customer_id': '13001', 'total_price': 1390.08}
{'transaction_id': '580680', 'customer_id': '13001', 'total_price': 21.72}
{'transaction_id': '580680', 'customer_id': '13001', 'total_price': 144.8}
{'transaction_id': '580680', 'customer_id': '13001', 'total_price': 86.88}
{'transaction_id': '580680', 'customer_id': '13001', 'total_price': 86.88}
{'transaction_id': '580680', 'customer_id': '13001', 'total_price': 43.44}
{'transaction_id': '580680', 'customer_id': '13001', 'total_price': 45.96}
{'transaction_id': '580680', 'customer_id': '13001', 'total_price': 91.92}
{'transaction_id': '580680', 'customer_id': '13001', 'total_price': 22.98}
{'transaction_id': '580680', 'customer_id': '13001', 'total_price': 24.52}
{'transaction_id': '580680', 'customer_id': '13

In [17]:
transaction_fact = pd.DataFrame(transformed_data)

In [18]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [19]:
transaction_fact.to_csv('Transaction_fact.csv')
!cp data.csv "drive/My Drive/"

cp: cannot stat 'data.csv': No such file or directory


**Dimension table - Product**

In [37]:
with open("product_table.csv", "r", encoding="utf-8-sig") as file:
    data = list(csv.DictReader(file))
    for item in data:
        item["ProductNo"] =str(item["ProductNo"])
data[:3]

[{'ProductNo': '22485', 'ProductName': 'Set Of 2 Wooden Market Crates'},
 {'ProductNo': '22596', 'ProductName': 'Christmas Star Wish List Chalkboard'},
 {'ProductNo': '23235', 'ProductName': 'Storage Tin Vintage Leaf'}]

In [38]:
class Database:
    def __init__(self):
        self.db = {
            "source": [],
            "destination": [],
            "transform": [],
            "mapping": []
        }
        #add_source
        self.add_source(1,"ProductNo","$.ProductNo","str",True)
        self.add_source(2,"ProductName","$.ProductName","str",True)

        #add_transform
        self.add_transform(1,'CLEAN_STRING')


        #add_destination
        self.add_destination(1,"product_id","product_id","str","n/a")
        self.add_destination(2,"product_name","product_name","str","n/a")

        #add_mapping
        self.add_mapping(1,1,1)
        self.add_mapping(2,2,2,1)

    def add_source(self,id,source_field_name,source_field_mapping,source_field_type,is_required):
      if not isinstance(is_required, bool):
        print("Error. Please enter if required.")
      else:
        source = {
            "id": id,
            "source_field_name": source_field_name,
            "source_field_mapping": source_field_mapping,
            "source_field_type": source_field_type,
            "is_required": is_required
        }
        self.db["source"].append(source)

    def add_transform(self,id,transform_mask):
      transform={
          "id": id,
          "transform_mask": transform_mask
      }
      self.db["transform"].append(transform)

    def add_destination(self,id,destination_field_name,destination_field_mapping,destination_field_type,default_value):
        destination = {
              "id": id,
              "destination_field_name": destination_field_name,
              "destination_field_mapping": destination_field_mapping,
              "destination_field_type": destination_field_type,
              "default_value": default_value
          }
        self.db["destination"].append(destination)

    def add_mapping(self,id,mapping_source,mapping_destination,mapping_transform=None):
      if mapping_transform is not None:
        mapping={
          "id": id,
          "mapping_source": mapping_source,
          "mapping_destination": mapping_destination,
          "mapping_transform": mapping_transform
        }
      else:
        mapping={
          "id": id,
          "mapping_source": mapping_source,
          "mapping_destination": mapping_destination
          }
      self.db["mapping"].append(mapping)

    @property
    def get_data_source_target_mapping(self):
        return self.db

In [39]:
transformed_data = []
for item in data:
    helper = STTM(input_json=item)
    response = helper.get_transformed_data()
    transformed_data.append(response)
    print(response)

{'product_id': '22485', 'product_name': 'set of 2 wooden market crates'}
{'product_id': '22596', 'product_name': 'christmas star wish list chalkboard'}
{'product_id': '23235', 'product_name': 'storage tin vintage leaf'}
{'product_id': '23272', 'product_name': 'tree t-light holder willie winkie'}
{'product_id': '23239', 'product_name': 'set of 4 knick knack tins poppies'}
{'product_id': '21705', 'product_name': 'bag 500g swirly marbles'}
{'product_id': '22118', 'product_name': 'joy wooden block letters'}
{'product_id': '22119', 'product_name': 'peace wooden block letters'}
{'product_id': '22217', 'product_name': 't-light holder hanging lace'}
{'product_id': '22216', 'product_name': 't-light holder white lace'}
{'product_id': '22380', 'product_name': 'toy tidy spaceboy'}
{'product_id': '22442', 'product_name': 'grow your own flowers set of 3'}
{'product_id': '22664', 'product_name': 'toy tidy dolly girl design'}
{'product_id': '22721', 'product_name': 'set of 3 cake tins sketchbook'}
{'p

In [40]:
product_dimension = pd.DataFrame(transformed_data)

In [41]:
product_dimension.to_csv('Product_dimension.csv')
!cp data.csv "drive/My Drive/"

cp: cannot stat 'data.csv': No such file or directory


**Dimension table - Customer**

In [42]:
with open("customer_table.csv", "r", encoding="utf-8-sig") as file:
    data = list(csv.DictReader(file))
    for item in data:
        item["CustomerNo"] =str(item["CustomerNo"])
data[:3]

[{'CustomerNo': '17490', 'Country': 'United Kingdom'},
 {'CustomerNo': '13069', 'Country': 'United Kingdom'},
 {'CustomerNo': '13069', 'Country': 'United Kingdom'}]

In [43]:
class Database:
    def __init__(self):
        self.db = {
            "source": [],
            "destination": [],
            "transform": [],
            "mapping": []
        }
        #add_source
        self.add_source(1,"CustomerNo","$.CustomerNo","str",True)
        self.add_source(2,"Country","$.Country","str",True)

        #add_transform
        self.add_transform(1,'CLEAN_STRING')


        #add_destination
        self.add_destination(1,"customer_id","customer_id","str","n/a")
        self.add_destination(2,"country","country","str","n/a")

        #add_mapping
        self.add_mapping(1,1,1)
        self.add_mapping(2,2,2,1)

    def add_source(self,id,source_field_name,source_field_mapping,source_field_type,is_required):
      if not isinstance(is_required, bool):
        print("Error. Please enter if required.")
      else:
        source = {
            "id": id,
            "source_field_name": source_field_name,
            "source_field_mapping": source_field_mapping,
            "source_field_type": source_field_type,
            "is_required": is_required
        }
        self.db["source"].append(source)

    def add_transform(self,id,transform_mask):
      transform={
          "id": id,
          "transform_mask": transform_mask
      }
      self.db["transform"].append(transform)

    def add_destination(self,id,destination_field_name,destination_field_mapping,destination_field_type,default_value):
        destination = {
              "id": id,
              "destination_field_name": destination_field_name,
              "destination_field_mapping": destination_field_mapping,
              "destination_field_type": destination_field_type,
              "default_value": default_value
          }
        self.db["destination"].append(destination)

    def add_mapping(self,id,mapping_source,mapping_destination,mapping_transform=None):
      if mapping_transform is not None:
        mapping={
          "id": id,
          "mapping_source": mapping_source,
          "mapping_destination": mapping_destination,
          "mapping_transform": mapping_transform
        }
      else:
        mapping={
          "id": id,
          "mapping_source": mapping_source,
          "mapping_destination": mapping_destination
          }
      self.db["mapping"].append(mapping)

    @property
    def get_data_source_target_mapping(self):
        return self.db

In [44]:
transformed_data = []
for item in data:
    helper = STTM(input_json=item)
    response = helper.get_transformed_data()
    transformed_data.append(response)
    print(response)

{'customer_id': '17490', 'country': 'united kingdom'}
{'customer_id': '13069', 'country': 'united kingdom'}
{'customer_id': '13069', 'country': 'united kingdom'}
{'customer_id': '13069', 'country': 'united kingdom'}
{'customer_id': '13069', 'country': 'united kingdom'}
{'customer_id': '13069', 'country': 'united kingdom'}
{'customer_id': '13069', 'country': 'united kingdom'}
{'customer_id': '13069', 'country': 'united kingdom'}
{'customer_id': '13069', 'country': 'united kingdom'}
{'customer_id': '13069', 'country': 'united kingdom'}
{'customer_id': '13069', 'country': 'united kingdom'}
{'customer_id': '13069', 'country': 'united kingdom'}
{'customer_id': '13069', 'country': 'united kingdom'}
{'customer_id': '13069', 'country': 'united kingdom'}
{'customer_id': '13069', 'country': 'united kingdom'}
{'customer_id': '13069', 'country': 'united kingdom'}
{'customer_id': '13069', 'country': 'united kingdom'}
{'customer_id': '13069', 'country': 'united kingdom'}
{'customer_id': '13069', 'co

In [46]:
customer_dimension = pd.DataFrame(transformed_data)

In [47]:
customer_dimension.to_csv('Customer_dimension.csv')
!cp data.csv "drive/My Drive/"

cp: cannot stat 'data.csv': No such file or directory
